In [35]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import requests

In [36]:
def fetch_market_indicators(start_date: datetime, end_date=None) -> pd.DataFrame:
    """
    Fetch market relationship indicators from Yahoo Finance.
    
    Parameters:
    start_date (str): Start date in 'YYYY-MM-DD' format
    end_date (str): End date in 'YYYY-MM-DD' format, defaults to today if None
    
    Returns:
    pandas.DataFrame: DataFrame containing market indicators
    """
    # If no end date specified, use today
    if end_date is None:
        end_date = datetime.today().strftime('%Y-%m-%d')
    
    # Define the symbols to fetch
    symbols = {
        # Original symbols
        'GC=F': 'Gold',
        'JPY=X': 'USD/JPY',
        '^GSPC': 'S&P 500',
        '^VIX': 'VIX',
        
        # New additions
        '^AXJO': 'ASX 200 (Australia)',
        '^FTSE': 'FTSE 100 (UK)',
        '^GDAXI': 'DAX (Germany)',
        'EUR=X': 'EUR/USD',
        'CL=F': 'Crude Oil',
        'SPY': 'S&P 500 ETF',
        '^NYHILO': 'NYSE New High/Low Index',
        
        # Additional bond ETFs
        'TLT': '20+ Year Treasury Bond ETF',
        # 'IEF': '7-10 Year Treasury Bond ETF',
        # 'SHY': '1-3 Year Treasury Bond ETF',
        'LQD': 'Investment Grade Corporate Bond ETF',
        # 'HYG': 'High Yield Corporate Bond ETF',
        
        # Additional currency pairs
        # 'GBPUSD=X': 'GBP/USD',
        # 'AUDUSD=X': 'AUD/USD',
        # 'CADUSD=X': 'CAD/USD',
        
        # Additional commodities
        # 'SI=F': 'Silver',
        'HG=F': 'Copper',
        # 'NG=F': 'Natural Gas'
    }
    
    # Initialize an empty dictionary to store the data
    data_dict = {}
    
    # Fetch data for each symbol
    for symbol, description in symbols.items():
        try:
            ticker = yf.Ticker(symbol)
            df = ticker.history(start=start_date, end=end_date)
            
            if not df.empty:
                # We'll use the adjusted closing price
                data_dict[symbol] = df['Close']
                print(f"Successfully fetched data for {symbols[symbol]} ({symbol})")
            else:
                print(f"No data available for {symbols[symbol]} ({symbol})")
        except Exception as e:
            print(f"Error fetching data for {symbol}: {str(e)}")
    
    # Combine all series into a single DataFrame
    combined_df = pd.DataFrame(data_dict)
    
    return combined_df

In [37]:
def generate_summary_stats(df):
    """
    Generate summary statistics for the dataset.
    
    Parameters:
    df (pandas.DataFrame): Input DataFrame
    
    Returns:
    pandas.DataFrame: Summary statistics
    """
    stats = pd.DataFrame({
        'start_date': df.index.min(),
        'end_date': df.index.max(),
        'data_points': df.count(),
        'missing_pct': (df.isna().sum() / len(df) * 100).round(2),
        'mean': df.mean().round(4),
        'std': df.std().round(4),
        'min': df.min().round(4),
        'max': df.max().round(4)
    }).T
    
    return stats

In [38]:

def fill_missing_data(df):
    """
    Fill missing data in financial time series using appropriate methods
    for different types of instruments.
    
    Parameters:
    df (pd.DataFrame): DataFrame with financial instruments and timestamp column 'Date'
    
    Returns:
    pd.DataFrame: Filled DataFrame
    """
    # Create a copy to avoid modifying the original
    df_filled = df.copy()
    
    # Convert timestamp to datetime
    df_filled['datetime'] = pd.to_datetime(df_filled.index, unit='ms')
    df_filled = df_filled.set_index('datetime')
    
    # Group columns by type
    market_indices = ['^GSPC', '^FTSE', '^GDAXI', '^AXJO']
    currencies = ['JPY=X', 'EUR=X']
    commodities = ['GC=F', 'CL=F', 'HG=F']
    etfs = ['SPY', 'TLT', 'LQD']
    
    # Fill market indices during their trading hours
    for idx in market_indices:
        df_filled[idx] = df_filled[idx].fillna(method='ffill', limit=8)
    
    # Interpolate currencies with time-weighted values
    for curr in currencies:
        df_filled[curr] = df_filled[curr].interpolate(method='time', limit=4)
    
    # Forward fill commodities but reset at day boundaries
    for comm in commodities:
        df_filled[comm] = df_filled.groupby(df_filled.index.date)[comm].fillna(method='ffill')
        df_filled[comm] = df_filled[comm].fillna(method='ffill')

    # Forward fill ETFs similar to their underlying indices
    for etf in etfs:
        df_filled[etf] = df_filled[etf].fillna(method='ffill', limit=8)
    
    # Special handling for VIX - use ffill with shorter window
    # Risks having stale VIX data into the future but that's the tradeoff
    df_filled['^VIX'] = df_filled['^VIX'].fillna(method='ffill')
    
    # Keep the original Date column
    df_filled['Date'] = df.index
    
    return df_filled

In [39]:
# Fetch the data
market_indicators_data = fetch_market_indicators(datetime.now() -  timedelta(days=1 * 365))
# fs = FeatureStorage('./Data/market_indicators.parquet')
# fs.save_features(market_indicators_data)


Successfully fetched data for Gold (GC=F)
Successfully fetched data for USD/JPY (JPY=X)
Successfully fetched data for S&P 500 (^GSPC)
Successfully fetched data for VIX (^VIX)
Successfully fetched data for ASX 200 (Australia) (^AXJO)
Successfully fetched data for FTSE 100 (UK) (^FTSE)
Successfully fetched data for DAX (Germany) (^GDAXI)
Successfully fetched data for EUR/USD (EUR=X)


$^NYHILO: possibly delisted; no timezone found


Successfully fetched data for Crude Oil (CL=F)
Successfully fetched data for S&P 500 ETF (SPY)
No data available for NYSE New High/Low Index (^NYHILO)
Successfully fetched data for 20+ Year Treasury Bond ETF (TLT)
Successfully fetched data for Investment Grade Corporate Bond ETF (LQD)
Successfully fetched data for Copper (HG=F)


In [40]:
market_indicators_data

,GC=F,JPY=X,^GSPC,^VIX,^AXJO,^FTSE,^GDAXI,EUR=X,CL=F,SPY,TLT,LQD,HG=F
Date,,,,,,,,,,,,,
2023-12-25 00:00:00+00:00,NaN,142.341995,NaN,NaN,NaN,NaN,NaN,0.90690,NaN,NaN,NaN,NaN,NaN
2023-12-26 00:00:00+00:00,NaN,142.229996,NaN,NaN,NaN,NaN,NaN,0.90742,NaN,NaN,NaN,NaN,NaN
2023-12-26 05:00:00+00:00,2058.199951,NaN,4774.750000,NaN,NaN,NaN,NaN,NaN,75.570000,469.625946,94.863472,105.614876,3.8955
2023-12-26 06:00:00+00:00,NaN,NaN,NaN,12.990000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-26 13:00:00+00:00,NaN,NaN,NaN,NaN,7561.200195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-19 06:00:00+00:00,NaN,NaN,NaN,24.090000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-12-19 23:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,19884.75,NaN,NaN,NaN,NaN,NaN,NaN
2024-12-20 00:00:00+00:00,NaN,157.643997,NaN,NaN,NaN,8084.600098,NaN,0.96479,NaN,NaN,NaN,NaN,NaN


In [41]:
# Generate and print summary statistics
summary_stats = generate_summary_stats(market_indicators_data)
print("\nSummary Statistics:")
print(summary_stats)
    
print("\nColumns in market indicators dataset:")
for col in market_indicators_data.columns:
    print(f"- {col}")


Summary Statistics:
                                  GC=F                      JPY=X  \
start_date   2023-12-25 00:00:00+00:00  2023-12-25 00:00:00+00:00   
end_date     2024-12-20 06:00:00+00:00  2024-12-20 06:00:00+00:00   
data_points                        250                        260   
missing_pct                      80.24                      79.45   
mean                         2379.3848                   151.0693   
std                           225.3726                     5.1156   
min                             1990.3                     140.79   
max                             2788.5                    161.621   

                                 ^GSPC                       ^VIX  \
start_date   2023-12-25 00:00:00+00:00  2023-12-25 00:00:00+00:00   
end_date     2024-12-20 06:00:00+00:00  2024-12-20 06:00:00+00:00   
data_points                        250                        250   
missing_pct                      80.24                      80.24   
mean        

In [42]:
market_indicators_data = fill_missing_data(market_indicators_data)

# Generate and print summary statistics
summary_stats = generate_summary_stats(market_indicators_data)
print("\nSummary Statistics:")
print(summary_stats)
    
print("\nColumns in market indicators dataset:")
for col in market_indicators_data.columns:
    print(f"- {col}")


Summary Statistics:
                                  GC=F                      JPY=X  \
start_date   2023-12-25 00:00:00+00:00  2023-12-25 00:00:00+00:00   
end_date     2024-12-20 06:00:00+00:00  2024-12-20 06:00:00+00:00   
data_points                       1263                       1265   
missing_pct                       0.16                        0.0   
mean                       2377.884474                 151.107039   
std                          225.03181                   5.052587   
min                        1990.300049                 140.789993   
max                             2788.5                 161.621002   

                                 ^GSPC                       ^VIX  \
start_date   2023-12-25 00:00:00+00:00  2023-12-25 00:00:00+00:00   
end_date     2024-12-20 06:00:00+00:00  2024-12-20 06:00:00+00:00   
data_points                       1263                       1262   
missing_pct                       0.16                       0.24   
mean        

/var/folders/kr/f61149ps2xl853qpvvmlwb040000gn/T/ipykernel_36044/206490385.py:27: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_filled[idx] = df_filled[idx].fillna(method='ffill', limit=8)
/var/folders/kr/f61149ps2xl853qpvvmlwb040000gn/T/ipykernel_36044/206490385.py:35: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fillna instead
  df_filled[comm] = df_filled.groupby(df_filled.index.date)[comm].fillna(method='ffill')
/var/folders/kr/f61149ps2xl853qpvvmlwb040000gn/T/ipykernel_36044/206490385.py:35: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_filled[comm] = df_filled.groupby(df_filled.index.date)[comm].fillna(method='ffill')
/var/folders/k

In [ ]:
market_indicators_data

,GC=F,JPY=X,^GSPC,^VIX,^AXJO,^FTSE,^GDAXI,EUR=X,CL=F,SPY,TLT,LQD,HG=F,Date
datetime,,,,,,,,,,,,,,
2024-11-22 00:00:00+00:00,2672.100098,154.218002,5948.709961,16.870001,8393.799805,8262.099609,19322.589844,0.955170,70.099998,591.678589,89.678925,107.543137,4.1130,2024-11-22 00:00:00+00:00
2024-11-22 05:00:00+00:00,2709.899902,154.213002,5969.339844,16.870001,8393.799805,8262.099609,19322.589844,0.955103,71.239998,593.512451,89.728561,107.642433,4.0740,2024-11-22 05:00:00+00:00
2024-11-22 06:00:00+00:00,2709.899902,154.212002,5969.339844,15.240000,8393.799805,8262.099609,19322.589844,0.955090,71.239998,593.512451,89.728561,107.642433,4.0740,2024-11-22 06:00:00+00:00
2024-11-24 13:00:00+00:00,2709.899902,154.156997,5969.339844,15.240000,8417.599609,8262.099609,19322.589844,0.954357,71.239998,593.512451,89.728561,107.642433,4.0740,2024-11-24 13:00:00+00:00
2024-11-24 23:00:00+00:00,2709.899902,154.146996,5969.339844,15.240000,8417.599609,8262.099609,19405.199219,0.954223,71.239998,593.512451,89.728561,107.642433,4.0740,2024-11-24 23:00:00+00:00
2024-11-25 00:00:00+00:00,2709.899902,154.145996,5969.339844,15.240000,8417.599609,8291.700195,19405.199219,0.954210,71.239998,593.512451,89.728561,107.642433,4.0740,2024-11-25 00:00:00+00:00
2024-11-25 05:00:00+00:00,2616.800049,154.194331,5987.370117,15.240000,8417.599609,8291.700195,19405.199219,0.954887,68.940002,595.525696,92.051437,108.982994,4.0915,2024-11-25 05:00:00+00:00
2024-11-25 06:00:00+00:00,2616.800049,154.203999,5987.370117,14.600000,8417.599609,8291.700195,19405.199219,0.955022,68.940002,595.525696,92.051437,108.982994,4.0915,2024-11-25 06:00:00+00:00
2024-11-25 13:00:00+00:00,2616.800049,154.271668,5987.370117,14.600000,8359.400391,8291.700195,19405.199219,0.955970,68.940002,595.525696,92.051437,108.982994,4.0915,2024-11-25 13:00:00+00:00
